<a href="https://colab.research.google.com/github/sazio/Transients/blob/master/Python/WLC_SSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import numpy as np
from tqdm.notebook import tqdm 

In [67]:
# Simulation's Parameters 
N_s = 588 # x_i with i in [1,...,N_s]
N_p = 10 # a_i with i in [1,...,N_p]
alpha = 1
beta = 2.5
V_1 = 0.9
eps = 0.01
sigma = 0.0001
tau = 480
V_0 = 2 

In [93]:
t = np.linspace(0, 10, num = 1000) # time steps 
a = np.asarray([np.ones(len(t)) for i in range(0,N_p)]) # principal neurons 
x = np.asarray([np.zeros(len(t)) for i in range(0,N_s)]) # sensory neurons

np.random.seed(137) # setting random seed  
eta = np.random.normal(0,0.0001, (N_s,N_p)) # small perturbations for initial state
proj_matrix = np.asarray([np.ones((N_s,N_p)) + eta  for i in range(0,len(t))]) # get shape
proj_matrix = np.moveaxis(proj_matrix, 0, -1) # shape  == (588, 10, 1000)

V = np.asarray([ V_0*np.ones((N_p, N_p)) - (V_0 - 1)*np.eye(N_p) for i in range(0, len(t))]) # competition matrix 
V = np.moveaxis(V, 0, -1)

(10, 10, 1000)

In [ ]:
delay = 4

for n in tqdm(range(0,len(t)-1)):
  for i in range(0, N_p):
    a[i, n+1] = (a[i,n]- a[i,n]*np.matmul(V[:,:,n],a[:,n]) + alpha*a[i, n]*np.matmul(proj_matrix[:,:,n].T, x[:, n])*(t[n+1] - t[n])
    for j in range(0, N_s):
      for k in range(0, N_s):
        P[i, j, n+1] = P[i, j, n] + eps*a[i, n]*(beta*x[j, n]-P[i, j, n])*(t[n+1] - t[n])
        V[i, j, n+1] = V[i, j, n] + eps*a[i, n]*a[k, n-delay]*(V_1 - V[i, j, n])*(t[n+1] - t[n])

In [59]:
def odeEuler(f,y0,t):
    '''Approximate the solution of y'=f(y,t) by Euler's method.

    Parameters
    ----------
    f : function
        Right-hand side of the differential equation y'=f(t,y), y(t_0)=y_0
    y0 : number
        Initial value y(t0)=y0 wher t0 is the entry at index 0 in the array t
    t : array
        1D NumPy array of t values where we approximate y values. Time step
        at each iteration is given by t[n+1] - t[n].

    Returns
    -------
    y : 1D NumPy array
        Approximation y[n] of the solution y(t_n) computed by Euler's method.
    '''
    y = np.zeros(len(t))
    y[0] = y0
    for n in range(0,len(t)-1):
      y[n+1] = y[n] + f(y[n],t[n])*(t[n+1] - t[n])
    return y